In [25]:
from jax.experimental import sparse
import jax.numpy as jnp
import numpy as np

In [134]:
def char_to_weight(character):
    if character == "x":
        return np.array([0, 1, 0, 0])
    if character == "y":
        return np.array([0, 0, 1, 0])
    if character == "z":
        return np.array([0, 0, 0, 1])
    if character == "i":
        return np.array([1, 0, 0, 0])
def char_to_index(character):
    if character == "x":
        return 1
    if character == "y":
        return 2
    if character == "z":
        return 3
    if character == "i":
        return 0
    
def group_instructorss_by_qubits(nested_list, n):
    grouped_data = []
    for sublist in nested_list:
        groups = {i: [] for i in range(n)}  # Create empty groups for all indices 0 to n
        for item in sublist:
            key = item[1]  # Second value in the tuple
            groups[key].append(item)
        grouped_data.append([groups[i] for i in range(n)])  # Append the lists in order
    return grouped_data

def mapper_noncx(character, instructors):
    # Map a single Pauliword to list
    # Example: xyy -> [0, 1, 0, 0][0,0,1,0][0,0,1,0] -- h --> [0, 0, 0, 1][0,0,-1,0][0,0,-1,0] = z(-y)(-y)
    from numpy import sin, cos, sqrt
    weights = char_to_weight(character)

    for gate, index, param in instructors:
        I, A, B, C = weights
        if gate == "h":
            weights = np.array([I, 0, 0, 1])
        if gate == "s":
            weights = np.array([I, 0, 1, 0])
        if gate == "t":
            weights = np.array([I, 1 / sqrt(2), 1 / sqrt(2), 0])
        if gate == "rx":
            weights = np.array([I, A, B * cos(param) - C * sin(param), B * sin(param) + C * cos(param)])
        if gate == "ry":
            weights = np.array([I, A * cos(param) + C * sin(param), B, C * cos(param) - A * sin(param)])
        if gate == "rz":
            weights = np.array([I, A * cos(param) - B * sin(param), B * cos(param) + A * sin(param), C])
    return weights

def construct_LUT_noncx(instructorsss, num_qubits):
    # instructorss has size k x n x [?], with k is number of non-cx layer, n is number of qubits, 
    # ? is the number of instructor.
    # lut has size k x n x 4 x 4, with 4 is the number of Pauliword, 4 for weights
    k = len(instructorsss)
    lut = np.zeros((k, num_qubits, 4, 4))
    print(lut.shape)
    characters = ["i", "x", "y", "z"]
    for k in range(k):
        for j in range(num_qubits):
            for i in range(4):
                lut[k][j][i] = mapper_noncx(characters[i], instructorsss[k][j])
    return lut

class Instructor:
    def __init__(self, num_qubits):
        self.clusters = []
        self.cluster = []
        self.cluster_temp = []
        self.xcluster = []
        self.xcluster_temp = []
        self.xclusters = []
        self.instructors = []
        self.num_qubits = num_qubits
        self.barriers = [0] * self.num_qubits
        self.is_cx_first = False
    def append(self, gate, index, param=0):
        self.instructors.append((gate, index, param))

    def clustering(self):
        if self.instructors[0][0] == "cx":
            self.is_cx_first = True
        self.barriers = [0] * self.num_qubits
        while len(self.instructors) > 0:
            gate, index, _ = self.instructors[0]
            
            is_break = False
            if gate == "cx":
                self.barriers[index[0]] += 1
                self.barriers[index[1]] += 1
                if sum(self.barriers) >= self.num_qubits and np.all(self.barriers):
                    if len(self.instructors) > 1:
                        if self.instructors[1][0] != "cx":
                            is_break = True

                self.xcluster.append(self.instructors.pop(0))
            else:
                if self.barriers[index] == 0:
                    self.cluster.append(self.instructors.pop(0))
                else:
                    self.cluster_temp.append(self.instructors.pop(0))
            if is_break:
                if len(self.cluster) > 0:
                    self.clusters.append(self.cluster)
                self.instructors =  self.cluster_temp + self.instructors
                if len(self.xcluster) > 0:
                    self.xclusters.append(self.xcluster)
                self.cluster = []
                self.cluster_temp = []
                self.xcluster = []
                self.barriers = [0] * self.num_qubits
                is_break = False
        if len(self.cluster) > 0:
            self.clusters.append(self.cluster)
        if len(self.cluster_temp) > 0:
            self.clusters.append(self.cluster_temp)
        if len(self.xcluster) > 0:
            self.xclusters.append(self.xcluster)
        return 


In [135]:
grouped_instructorss = group_instructorss_by_qubits(ins.clusters, ins.num_qubits)
print(len(grouped_instructorss))
print(len(grouped_instructorss[0]))
LUT = construct_LUT_noncx(grouped_instructorss, ins.num_qubits)

2
4
(2, 4, 4, 4)


In [136]:
LUT

array([[[[ 1.        ,  0.        ,  0.        ,  1.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ]],

        [[ 1.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  1.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.71091354,  0.70327942],
         [ 0.        ,  0.        , -0.70327942,  0.71091354]],

        [[ 1.        ,  0.        ,  0.        ,  1.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ]],

        [[ 1.        ,  0.        ,  0.        ,  1.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ],
         [ 0.        ,  0.        ,  0.        ,  1.        ],
         [ 0.        ,  0.        ,  0.        , 

In [128]:

ins = Instructor(4)
ins.append("h", 0)
ins.append("rx", 1, 0.78)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("ry", 0, 0.56)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.clustering()

In [91]:
ins.clusters

[[('h', 0, 0),
  ('h', 1, 0),
  ('h', 2, 0),
  ('h', 0, 0),
  ('h', 2, 0),
  ('h', 2, 0),
  ('h', 3, 0),
  ('h', 3, 0),
  ('h', 3, 0),
  ('h', 3, 0),
  ('h', 3, 0)],
 [('h', 0, 0),
  ('h', 1, 0),
  ('h', 0, 0),
  ('h', 1, 0),
  ('h', 2, 0),
  ('h', 0, 0),
  ('h', 2, 0)]]

In [69]:
ins.xclusters

[[['cx', [0, 1], 0], ['cx', [1, 2], 0], ['cx', [1, 3], 0]]]

In [80]:
num_qubits = 3
ins = Instructor(num_qubits)
for i in range(num_qubits - 1):
    ins.append('cx', [i, i + 1])
ins.append('cx', [num_qubits - 1, 0])
for i in range(num_qubits):
    ins.append('h', i)
for i in range(num_qubits - 1):
    ins.append('cx', [i, i + 1])
ins.append('cx', [num_qubits - 1, 0])
for i in range(num_qubits):
    ins.append('h', i)
ins.clustering()

In [81]:
ins.clusters

[[['h', 0, 0], ['h', 1, 0], ['h', 2, 0]],
 [['h', 0, 0], ['h', 1, 0], ['h', 2, 0]]]

In [72]:
ins.xclusters

[[['cx', [0, 1], 0], ['cx', [1, 2], 0], ['cx', [2, 0], 0]],
 [['cx', [0, 1], 0], ['cx', [1, 2], 0], ['cx', [2, 0], 0]]]